In [16]:
import couchdb
import yaml

In [2]:
SECRET_KEY = ""
with open('couch_key.yaml', 'r') as file:
    key = yaml.safe_load(file)
    SECRET_KEY = key["SECRET_KEY"]

In [26]:
server = couchdb.Server(f'http://{SECRET_KEY}@172.26.135.50:5984/')

In [6]:
db = server.create('python-tests')

In [7]:
from datetime import datetime
from couchdb.mapping import Document, TextField, IntegerField, DateTimeField
class Person(Document):
    name = TextField()
    age = IntegerField()
    added = DateTimeField(default=datetime.now)
person = Person(name='John Doe', age=42)
person.store(db)

<Person '0bde37bfc9aefdfe2c54c35886000fb5'@'1-1b311da960f4f46fc7d99dbc09bd2630' {'name': 'John Doe', 'age': 42, 'added': '2023-05-10T11:30:03.514930Z'}>

In [8]:
person.age

42

In [9]:
person = Person.load(db, person.id)
old_rev = person.rev
person.name
person.age
person.added   

datetime.datetime(2023, 5, 10, 11, 30, 3, 514930)

In [10]:
person.name = 'John R. Doe'
person.store(db)    

<Person '0bde37bfc9aefdfe2c54c35886000fb5'@'2-997107abac93398cedfeb4a204542bba' {'name': 'John R. Doe', 'age': 42, 'added': '2023-05-10T11:30:03.514930Z'}>

In [11]:
person = Person.load(db, person.id)
person.name
person.rev != old_rev

True

https://couchdb-python.readthedocs.io/en/latest/mapping.html

In [47]:
from couchdb.mapping import TextField
from couchdb.mapping import FloatField
from couchdb.mapping import IntegerField
from couchdb.mapping import LongField
from couchdb.mapping import BooleanField
from couchdb.mapping import DecimalField
from couchdb.mapping import DateField
from couchdb.mapping import DictField
from couchdb.mapping import ListField
from couchdb.mapping import ViewField
from couchdb.mapping import Mapping

In [20]:
from datetime import date

### class couchdb.mapping.DateField(name=None, default=None)
Mapping field for storing dates.

In [21]:
field = DateField()

print(field._to_python('2007-04-01'))

print(field._to_json(date(2007, 4, 1)))

print(field._to_json(datetime(2007, 4, 1, 15, 30)))


2007-04-01
2007-04-01
2007-04-01


### class couchdb.mapping.DateTimeField(name=None, default=None)
Mapping field for storing date/time values.

In [23]:
field = DateTimeField()
print(field._to_python('2007-04-01T15:30:00Z'))

print(field._to_python('2007-04-01T15:30:00.009876Z'))

print(field._to_json(datetime(2007, 4, 1, 15, 30, 0)))

print(field._to_json(datetime(2007, 4, 1, 15, 30, 0, 9876)))

print(field._to_json(date(2007, 4, 1)))

2007-04-01 15:30:00
2007-04-01 15:30:00.009876
2007-04-01T15:30:00Z
2007-04-01T15:30:00.009876Z
2007-04-01T00:00:00Z


### class couchdb.mapping.DictField(mapping=None, name=None, default=None)
Field type for nested dictionaries.

In [28]:
from couchdb import Server

if 'python-tests' not in server:
    db = server.create('python-tests')
else:
    db = server['python-tests']

In [35]:
class Post(Document):
    title = TextField()
    content = TextField()
    author = DictField(Mapping.build(
        name = TextField(),
        email = TextField()
    ))
    extra = DictField()

In [37]:
post = Post(
    title='Foo bar',
    author=dict(name='John Doe',
                email='john@doe.com'),
    extra=dict(foo='bar'),
)
post.store(db)

post = Post.load(db, post.id)
print(post.author.name)
print(post.author.email)
print(post.extra)


John Doe
john@doe.com
{'foo': 'bar'}


In [38]:
del server['python-tests']

### class couchdb.mapping.ListField(field, name=None, default=None)
Field type for sequences of other fields.

In [39]:
from couchdb import Server

if 'python-tests' not in server:
    db = server.create('python-tests')
else:
    db = server['python-tests']

In [43]:
class Post(Document):
    title = TextField()
    content = TextField()
    pubdate = DateTimeField(default=datetime.now)
    comments = ListField(DictField(Mapping.build(
        author = TextField(),
        content = TextField(),
        time = DateTimeField()
    )))

In [44]:
post = Post(title='Foo bar')
post.comments.append(author='myself', content='Bla bla',
                     time=datetime.now())
len(post.comments)

post.store(db) 

post = Post.load(db, post.id)
comment = post.comments[0]
print(comment['author'])

print(comment['content'])

print(comment['time'])

print(post.comments)

myself
Bla bla
2023-05-10T11:49:53.944498Z
[{'author': 'myself', 'content': 'Bla bla', 'time': '2023-05-10T11:49:53.944498Z'}]


In [45]:
del server['python-tests']

### class couchdb.mapping.ViewField(design, map_fun, reduce_fun=None, name=None, language='javascript', wrapper=<object object>, **defaults) 
</br>
Descriptor that can be used to bind a view definition to a property of a Document class.

In [48]:
class Person(Document):
    name = TextField()
    age = IntegerField()
    by_name = ViewField('people', '''\
        function(doc) {
            emit(doc.name, doc);
        }''')
Person.by_name

<ViewDefinition '_design/people/_view/by_name'>

In [49]:
print(Person.by_name.map_fun)

function(doc) {
    emit(doc.name, doc);
}


In [54]:
Person.by_name(db, count=3)

<ViewResults <PermanentView '_design/people/_view/by_name'> {'count': 2}>

In [51]:
class Person(Document):
    name = TextField()
    age = IntegerField()

    @ViewField.define('people')
    def by_name(doc):
        yield doc['name'], doc

In [52]:
Person.by_name

<ViewDefinition '_design/people/_view/by_name'>

In [53]:
print(Person.by_name.map_fun)

def by_name(doc):
    yield doc['name'], doc
